In [1]:
#installing transformers
#!pip install --upgrade transformers
#!pip install simpletransformers


In [2]:
import pandas as pd 
import numpy as np 
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [3]:
#Loading the Training Text and label Data

#Text data
train_text = open('/content/train_text.txt', 'r')
tweet_train_text = train_text.readlines()
print(tweet_train_text[:20])

#Label data 
train_label = open('/content/train_labels.txt', 'r')
tweet_train_labels = train_label.readlines()
print(tweet_train_labels[:20])

['seeing ppl walking w/ crutches makes me really excited for the next 3 weeks of my life  \n', 'look for the girl with the broken smile, ask her if she wants to stay while, and she will be loved. 💕🎵 \n', 'Now I remember why I buy books online @user #servicewithasmile  \n', '@user @user So is he banded from wearing the clothes?  #Karma \n', 'Just found out there are Etch A Sketch apps.  #oldschool #notoldschool \n', "Hey what do you know, one of the witnesses supporting Darren Wilson's story lied! And is racist! Mind blown!  \n", '@user on stage at #flzjingleball at the @user in #Tampa #iheartradio \n', "You know it's going to be a great day when you're Garmin resets itself and you spill some cinnamon down yourself  #slowclap \n", 'Halfway thorough my workday ... Woooo  \n', 'Would like to thank my nephew for giving me his horrible cold & sore throat etc.. Much appreciated!  \n', "I forked node!  Get ready for the future.  (Where's my interviews)  \n", "@user @user @user @user I'm off t

In [4]:
#Loading the Test Text and label Data

#Text data
test_text = open('/content/test_text.txt', 'r')
tweet_test_text = test_text.readlines()
print(tweet_test_text[:20])

#Label data 
test_label = open('/content/test_labels.txt', 'r')
tweet_test_labels = test_label.readlines()
print(tweet_test_labels[:20])

['@user Can U Help?||More conservatives needed on #TSU + get paid 4 posting stuff like this!||YOU $ can go to \n', 'Just walked in to #Starbucks and asked for a "tall blonde" Hahahaha #irony \n', '#NOT GONNA WIN \n', '@user He is exactly that sort of person. Weirdo! \n', "So much #sarcasm at work mate 10/10 #boring 100% #dead mate full on #shit absolutely #sleeping mate can't handle the #sarcasm \n", 'Corny jokes are my absolute favorite \n', 'People complain about my backround pic and all I feel is like "hey don\'t blame me, Albert E might have spoken those words" #sarcasm #life \n', '@user @user Darn, my sock joke needs fixing? \n', "if Christian expects Fifa to sleep in my bed with me tonight, he's wrong 👿 \n", 'People who tell people with anxiety to "just stop worrying about it" are my favorite kind of people #not #educateyourself \n', "Most important thing I've learned in school \n", '#OrionLaunch underway - yeah! #Orion \n', 'I love context and large ensemble Fridays!!!!! Der my 

In [5]:
#converting label data into integers
def preprocess_labels(labels):
  train_int_labels = []
  for n in labels:
    n = n.replace('\n', '')
    n = int(n)
    train_int_labels.append(n)
  return train_int_labels

preprocessed_labels_train = preprocess_labels(tweet_train_labels)
preprocessed_labels_train[:20]


[1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0]

In [6]:
#preprocessing tweet text 


def preprocess_text(text):
    processed_train_text = []
    for t in text:
        t = t.replace('@user', '')
        t = t.replace('\n', '')
        t = 'http' if t.startswith('http') else t
        processed_train_text.append(t)
    return processed_train_text

preprocessed_train = preprocess_text(tweet_train_text)
preprocessed_train[:20]

['seeing ppl walking w/ crutches makes me really excited for the next 3 weeks of my life  ',
 'look for the girl with the broken smile, ask her if she wants to stay while, and she will be loved. 💕🎵 ',
 'Now I remember why I buy books online  #servicewithasmile  ',
 '  So is he banded from wearing the clothes?  #Karma ',
 'Just found out there are Etch A Sketch apps.  #oldschool #notoldschool ',
 "Hey what do you know, one of the witnesses supporting Darren Wilson's story lied! And is racist! Mind blown!  ",
 ' on stage at #flzjingleball at the  in #Tampa #iheartradio ',
 "You know it's going to be a great day when you're Garmin resets itself and you spill some cinnamon down yourself  #slowclap ",
 'Halfway thorough my workday ... Woooo  ',
 'Would like to thank my nephew for giving me his horrible cold & sore throat etc.. Much appreciated!  ',
 "I forked node!  Get ready for the future.  (Where's my interviews)  ",
 "    I'm off to visit great-nephew very ill, only 20  fair ",
 'If his

In [7]:
df_train = pd.DataFrame({'text': preprocessed_train,'label': preprocessed_labels_train})
df_train

,text,label
0,seeing ppl walking w/ crutches makes me really...,1
1,"look for the girl with the broken smile, ask h...",0
2,Now I remember why I buy books online #servic...,1
3,So is he banded from wearing the clothes? #...,1
4,Just found out there are Etch A Sketch apps. ...,1
...,...,...
2857,I don't have to respect your beliefs.||I only ...,0
2858,Women getting hit on by married managers at c...,1
2859,no but i followed you and i saw you posted th...,0
2860,I dont know what it is but I'm in love your p...,0


In [8]:
#checking length of train and labels 
print(len(preprocessed_train), len(preprocessed_labels_train))

2862 2862


In [9]:
#using model from simple transform 

model_args = ClassificationArgs(num_train_epochs=1, overwrite_output_dir=True, manual_seed=42)
model = ClassificationModel(model_type='xlnet', model_name='xlnet-base-cased', use_cuda=True, num_labels=2, args=model_args)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [10]:
#training model
model.train_model(df_train)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:446: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/358 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


(358, 0.6798991973007191)

In [11]:
#preprocessing test data 
preprocessed_labels_test = preprocess_labels(tweet_test_labels)
preprocessed_text_test = preprocess_text(tweet_test_text)

df_test = pd.DataFrame({'text': preprocessed_text_test,'label': preprocessed_labels_test})
df_train

,text,label
0,seeing ppl walking w/ crutches makes me really...,1
1,"look for the girl with the broken smile, ask h...",0
2,Now I remember why I buy books online #servic...,1
3,So is he banded from wearing the clothes? #...,1
4,Just found out there are Etch A Sketch apps. ...,1
...,...,...
2857,I don't have to respect your beliefs.||I only ...,0
2858,Women getting hit on by married managers at c...,1
2859,no but i followed you and i saw you posted th...,0
2860,I dont know what it is but I'm in love your p...,0


In [12]:
print(len(preprocessed_labels_test), len(preprocessed_text_test))

784 784


In [13]:
result, model_outputs, wrong_predictions = model.eval_model(df_test)
result, model_outputs, wrong_predictions

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:1026: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Running Evaluation:   0%|          | 0/98 [00:00<?, ?it/s]

({'auprc': 0.582626546447615,
  'auroc': 0.7117393934861967,
  'eval_loss': 0.6355718356005999,
  'fn': 67,
  'fp': 218,
  'mcc': 0.3218659082984271,
  'tn': 255,
  'tp': 244},
 array([[-0.41821289, -1.02636719],
        [-0.96337891, -0.47583008],
        [ 0.0670166 , -1.52441406],
        ...,
        [-1.34960938,  0.13635254],
        [-0.02282715, -1.35253906],
        [ 0.27270508, -1.80761719]]),
  <simpletransformers.classification.classification_utils.InputExample at 0x7fcb82c14d68>])

In [14]:
#predictions 

predictions, raw_outputs = model.predict(df_test['text'])
predictions

  0%|          | 0/98 [00:00<?, ?it/s]

array([0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1,

In [15]:
test_pred = []
for pred in predictions:
  pred = str(pred)
  test_pred.append(pred)
test_pred = '\n'.join(test_pred)
test_pred = test_pred.replace('[', '')
test_pred = test_pred.replace(']', '')
test_pred 

'0\n1\n0\n1\n0\n1\n1\n0\n1\n1\n1\n0\n1\n1\n1\n0\n0\n0\n1\n0\n0\n0\n1\n0\n0\n0\n0\n1\n0\n0\n0\n1\n1\n0\n1\n0\n0\n0\n1\n1\n1\n1\n1\n1\n1\n0\n1\n1\n1\n1\n1\n0\n0\n1\n1\n1\n1\n1\n0\n0\n1\n1\n1\n1\n1\n0\n0\n0\n0\n1\n1\n1\n1\n1\n0\n1\n0\n0\n0\n0\n0\n0\n0\n0\n1\n0\n1\n1\n0\n0\n1\n0\n0\n1\n1\n1\n1\n1\n0\n1\n1\n1\n0\n1\n1\n1\n1\n0\n0\n0\n1\n0\n0\n1\n1\n1\n0\n1\n0\n0\n0\n0\n1\n0\n0\n1\n1\n0\n1\n0\n1\n1\n1\n1\n0\n1\n0\n0\n1\n1\n0\n0\n0\n0\n0\n1\n1\n0\n0\n0\n1\n1\n0\n1\n0\n1\n1\n1\n0\n0\n1\n1\n0\n1\n0\n1\n0\n1\n0\n1\n1\n1\n1\n0\n1\n1\n0\n1\n1\n1\n1\n1\n0\n0\n1\n0\n1\n0\n1\n1\n1\n1\n0\n1\n1\n0\n1\n1\n1\n1\n0\n0\n0\n1\n1\n0\n0\n1\n1\n0\n0\n1\n1\n0\n0\n1\n0\n1\n1\n0\n1\n1\n1\n1\n0\n1\n1\n1\n1\n0\n1\n0\n0\n1\n0\n0\n0\n1\n0\n1\n1\n0\n0\n1\n1\n1\n0\n1\n0\n1\n0\n1\n0\n1\n0\n1\n0\n0\n0\n1\n0\n0\n0\n0\n1\n0\n0\n1\n0\n1\n0\n0\n1\n1\n1\n1\n1\n1\n1\n1\n0\n0\n1\n0\n0\n1\n0\n0\n1\n1\n1\n0\n1\n1\n1\n1\n1\n0\n0\n1\n1\n1\n1\n0\n0\n1\n1\n1\n1\n0\n1\n0\n1\n0\n1\n1\n0\n0\n0\n1\n0\n1\n1\n0\n0\n0\n1\n1\n1\n0\n1\n1\n0\n

In [16]:
irony_pred_file = open("irony.txt", "w")
irony_pred_file.write(test_pred)
irony_pred_file.close()

In [19]:
#using Tweet Eval evaluation metric 

%run evaluation_script.py --tweeteval_path ./ --predictions_path ./ --task irony

0 {'precision': 0.7912772585669782, 'recall': 0.538135593220339, 'f1-score': 0.6406052963430012, 'support': 472}
1 {'precision': 0.5281385281385281, 'recall': 0.7845659163987139, 'f1-score': 0.6313065976714102, 'support': 311}
accuracy 0.6360153256704981
macro avg {'precision': 0.6597078933527531, 'recall': 0.6613507548095264, 'f1-score': 0.6359559470072057, 'support': 783}
weighted avg {'precision': 0.6867611089332004, 'recall': 0.6360153256704981, 'f1-score': 0.6369119434862134, 'support': 783}
------------------------------
TweetEval Score (irony): 0.6313065976714102
